In [8]:
import re

import requests
from pyquery import PyQuery as pq

In [9]:
r = requests.get("https://www.pa.gov/agencies/dli/programs-services/workforce-development-home/warn-requirements/warn-notices")

In [10]:
html = r.content

In [11]:
entries = pq(html)("div.cmp-accordion__item")

In [12]:
# NOT USED. NOT FINISHED. Mostly implemented below.

def clean_deets(deets: str):
    deets = "COUNT" + "COUNT".join(blob.split("COUNT")[1:])   # Split on COUNTY/COUNTIES
    deets = deets.replace("\xa0", " ")     # Fake spaces or non-breaking spaces show up
    deets = deets.replace("# AFFECTED", "AFFECTED")
    if deets.beginswith("COUNTIES"):
        deets = deets.replace("COUNTIES", "COUNTY")   # Replace anywhere. This can't go wrong.
    lines = deets.split("\n")
    better = []
    betteri = 0
    # for linei, line in enumerate(lines):
        

In [17]:
masterlist = []
for entry in entries:
    if not "cmp-accordion__item" in pq(entry).html():   # Fin items not holing other items
        line = {}
        line['Company'] = pq(entry)("span.cmp-accordion__title").text().strip()
        blob = pq(entry)("div.text").text()
        # address = pq(pq(blob)("p")[0]).text()
        # addressfull = ", ".join(address.split("\n"))
        address = blob.split("COUNT")[0].strip()
        deets = "COUNT" + "COUNT".join(blob.split("COUNT")[1:]).replace("\xa0", " ").replace("# AFFECTED", "AFFECTED")
        deets = deets.split("\n")
        line['addressfull'] = address.replace("\n", ", ")
        lastkey = None
        for deet in deets:
            if "PHASE" in deet.upper():
                if len(line[lastkey]) > 0:
                    line[lastkey] += " ... "
                line[lastkey] += deet.strip()                
            elif ":" in deet and "Ending:" not in deet:
                key = deet.split(":")[0]
                if key == "COUNTIES":
                    key = "COUNTY"
                value = ": ".join(deet.split(":")[1:]).strip()
                line[key] = value
                lastkey = key
            else:   # No colon in deet and no Phase
                if len(line[lastkey]) > 0:
                    line[lastkey] += " ... "
                line[lastkey] += deet.strip()
            
        masterlist.append(line)

In [18]:
crosswalk = {
    "COUNTIES": "county",
    "COUNTY": "county",
    "Company": "company",
    "addressfull": "addressfull",
    "AFFECTED": "jobs",
    "EFFECTIVE DATE": "date_effective",
    "EFFECTIVE DATES": "date_effective",
    "CLOSURE OR LAYOFF": "closure_or_layoff",
}

In [19]:
tallies = {}
for row in masterlist:
    for entry in row:
        if entry not in tallies:
            tallies[entry] = 0
        tallies[entry] += 1
for entry in tallies:
    print(f"{entry}: {tallies[entry]}")

Company: 231
addressfull: 231
COUNTY: 231
AFFECTED: 231
EFFECTIVE DATE: 190
CLOSURE OR LAYOFF: 231
EFFECTIVE DATES: 28


In [20]:
templist = masterlist
masterlist = []
for row in templist:
    line = {}
    for item in row:
        if item not in crosswalk:
            print(f"Failed to find a matching crosswalk term for {item}")
        else:
            line[crosswalk[item]] = row[item]
    masterlist.append(line)

In [ ]:
# multiple ates an aresses:
print(masterlist[200])

In [ ]:
print(pq(entry))

In [ ]:
print(pq(blob).text())

In [ ]:
len(masterlist)